In [1]:
import pandas as pd

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/laptops-info/Final_Dataframe.csv


In [2]:
data = pd.read_csv('/kaggle/input/laptops-info/Final_Dataframe.csv', sep=',' ,encoding='latin-1', usecols=['laptop_name', 'processor_type'])

In [3]:
data.head(1)

,laptop_name,processor_type
0,Notebook 14-df0008nx,Intel Celeron N4000


In [4]:
data['spec'] = data['processor_type']
data.head(1)

,laptop_name,processor_type,spec
0,Notebook 14-df0008nx,Intel Celeron N4000,Intel Celeron N4000


In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(token_pattern = '[a-zA-Z0-9\-]+')
tfidf_matrix = tfidf.fit_transform(data['spec'])
tfidf_matrix.shape

(205, 31)

In [6]:
from sklearn.metrics.pairwise import linear_kernel

cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
cosine_sim.shape

(205, 205)

In [7]:
def recommendations(title, cosine_sim = cosine_sim):
    idx = data.loc[data['laptop_name'].isin([title])]
    idx = idx.index
    sim_scores = list(enumerate(cosine_sim[idx][0]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:3]
    indices = [i[0] for i in sim_scores]
    return data['laptop_name'].iloc[indices]

In [8]:
x = 'IdeaPad 330'
result = recommendations(x)
result = result.values.tolist()
for hasil in result:
    print(hasil)

Vostro 5471
Swift 5 SF514-52TP-8933


In [9]:
from sklearn.neighbors import KNeighborsClassifier

def get_label(id):
    classifier = KNeighborsClassifier(n_neighbors=5)
    x = tfidf_matrix
    y = data.iloc[:,-1]
    classifier.fit(x, y)
    y_pred = classifier.predict(tfidf_matrix[id])
    return y_pred

In [10]:
true_count = 0
false_count = 0
def evaluate_content_based_model():
    for key, colums in data.iterrows():
        recommended_by_model = recommendations(colums["laptop_name"])
        predicted  = get_label(recommended_by_model.index)
        for predicteds in predicted:
            global true_count, false_count
            if predicteds == colums["spec"]:
                true_count = true_count+1
            else:
                false_count = false_count +1
evaluate_content_based_model()
total = true_count + false_count
print("Hit:"+ str(true_count/total))
print("Fault:" + str(false_count/total))

Hit:0.7317073170731707
Fault:0.2682926829268293
